In [1]:
import pandas as pd
import numpy as np

pd.reset_option('display.max_rows')

In [2]:
df = pd.read_excel("lr_output.xlsx")

In [10]:
def display_result(stakes_sum, profit_sum, bet_num, betting_yield, func_name, win_count, lose_count, win_rate):
    # Display the result
    print(func_name.upper() + ':')
    print("=========================")
    print("Stakes sum:", round(stakes_sum,1))
    print("Net profit:", round(profit_sum,1))
    print("Yield:     ", round(betting_yield,3), "%")
    print("Total bets:", bet_num)
    print("Bets won:  ", win_count)
    print("Bets lost: ", lose_count)
    print("Win rate:  ", round(win_rate,3), '%')
    print("=========================")
    #return [total, profit, roi, bet_num]

In [11]:
def process_result(df, func, betting_unit, par2=0, par3=0, save=0):
    # Náhodné zamíchání řádků v datasetu
    shuffled_df = df.sample(frac=1).reset_index(drop=True)

    # Zavolání konkrétní sázkové strategie
    func_output = func(shuffled_df, betting_unit, par2, par3)

    # Tvorba sloupce pro vypočítaný profit každé sázky
    shuffled_df['profit'] = func_output[0]

    # Vytvoření proměnných pro vypsání výsledků
    func_name = func_output[1]
    win_count = func_output[2]
    lose_count = func_output[3]
    win_rate = (win_count / (win_count + lose_count)) * 100
    bet_num = len(df) - np.count_nonzero(shuffled_df['profit'] == 0)
    stakes_sum = bet_num * betting_unit

    # Výpočet celkové sumy profitu
    profit_sum = shuffled_df['profit'].sum()

    # výpočet výtěžnosti sázení
    betting_yield = (profit_sum / stakes_sum) * 100

    # pokud je parametr funkce save==1, uložit výsledek do excelu
    if save == 1:
        shuffled_df.to_excel(f"betting_output/{func_name}.xlsx")

    # zavolání funkce display_result pro vypsání výsledku
    display_result(stakes_sum, profit_sum, bet_num, betting_yield, func_name, win_count, lose_count, win_rate)

In [345]:
def process_martingale_test(df, func, betting_unit, par2=0, par3=0, save=0, iter_num=100):
    x1, x2, x3, x4, x5, x6, x7 = 0, 0, 0, 0, 0, 0, 0   
    for i in range(iter_num):
        # Náhodné zamíchání řádků v datasetu
        shuffled_df = df.sample(frac=1).reset_index(drop=True)

        # Zavolání konkrétní sázkové strategie
        func_output = func(shuffled_df, betting_unit, par2, par3)

        # Tvorba sloupce pro vypočítaný profit každé sázky
        shuffled_df['profit'] = func_output[0]

        # Vytvoření proměnných pro vypsání výsledků
        win_count = func_output[2]
        lose_count = func_output[3]
        win_rate = (win_count / (win_count + lose_count)) * 100
        bet_num = len(df) - np.count_nonzero(shuffled_df['profit'] == 0)
        stakes_sum = bet_num * betting_unit

        # Výpočet celkové sumy profitu
        profit_sum = shuffled_df['profit'].sum()

        # výpočet výtěžnosti sázení
        betting_yield = (profit_sum / stakes_sum) * 100

        # Při každé iteraci připočte nové hodnoty do promenných x1-7
        x1 += stakes_sum
        x2 += profit_sum
        x3 += bet_num
        x4 += betting_yield
        x5 += win_count
        x6 += lose_count
        x7 += win_rate
    
    # Výpočet průměru hodnot vydělením celkového počtu iterací od hodnot x1-7 
    func_name = func_output[1] + '_(AVG_OF_100_RUNS)'
    avg_stakes_sum = x1 / iter_num
    avg_profit_sum = x2 / iter_num
    avg_bet_num = x3 / iter_num
    avg_betting_yield = x4 / iter_num
    avg_win_count = x5 / iter_num
    avg_lose_count = x6 / iter_num
    avg_win_rate = x7 / iter_num

    # zavolání funkce display_result pro vypsání výsledku
    display_result(avg_stakes_sum, avg_profit_sum, avg_bet_num, avg_betting_yield, func_name, avg_win_count, avg_lose_count, avg_win_rate)

In [356]:
def bm_probability_fixed(df, betting_unit, odds_limit, arb=0):
    profit_lst = []
    odds_lst = ['pl2_avg_bookmaker_odds', 'pl1_avg_bookmaker_odds']
    win_count = 0
    lose_count = 0
    for index, row in df.iterrows():
        # Pokud je jeden z vypsaných kurzu menší nebo roven odds_limit
        if row[odds_lst[0]] <= odds_limit or row[odds_lst[1]] <= odds_limit:
            for i in range(2):
                if i == 0:
                    pl = 1
                else:
                    pl = 0
                if row[odds_lst[i]] <= odds_limit:
                    if row['bookmaker_result'] == i and row['result'] == pl:
                        profit_lst.append(-betting_unit)
                        lose_count += 1
                    elif row['bookmaker_result'] == i and row['result'] == i:
                        profit = (betting_unit*row[odds_lst[i]])-betting_unit
                        profit_lst.append(profit)
                        win_count += 1
                    else:
                        profit = (betting_unit*row[odds_lst[i]])-betting_unit
                        profit_lst.append(profit)
                        win_count += 1
                else:
                    continue
        else:
            profit_lst.append(0)
    return [profit_lst, 'bm_probability_fixed', win_count, lose_count]

In [3]:
def bm_probability_martingale(df, betting_unit, arb1=0, arb2=0):
    profit_lst = []
    original_betting_unit = betting_unit
    win_count = 0
    lose_count = 0
    pl_odds = ['pl2_avg_bookmaker_odds', 'pl1_avg_bookmaker_odds']
    for index, row in df.iterrows():
        if ((row['pl1_avg_bookmaker_odds'] < row['pl2_avg_bookmaker_odds']+0.2) 
        and (row['pl1_avg_bookmaker_odds'] > row['pl2_avg_bookmaker_odds']-0.2)):
            if row['bookmaker_result'] != row['result']:
                profit_lst.append(-betting_unit)
                betting_unit *= 2
                lose_count += 1
            else:
                odds = pl_odds[row['result']]
                profit_lst.append((betting_unit*row[f'{odds}'])-betting_unit)
                betting_unit = original_betting_unit
                win_count += 1
        else:
            profit_lst.append(0)
    return [profit_lst, 'bm_probability_martingale', win_count, lose_count]

In [1]:
def lr_probability_fixed(df, betting_unit, prob_limit, odds_limit):
    profit_lst = []
    odds_lst = ['pl2_avg_bookmaker_odds', 'pl1_avg_bookmaker_odds']
    win_count = 0
    lose_count = 0
    for index, row in df.iterrows():
        # Výběr vítěze
        pl1_bet = row['pl1_prob'] >= 1-prob_limit
        pl2_bet = row['pl1_prob'] <= prob_limit
        pl_bet_lst = [pl2_bet, pl1_bet]
        # Ověření podmínky prob_limit a odds_limit
        if ((pl1_bet and row[odds_lst[1]] > odds_limit) 
        or (pl2_bet and row[odds_lst[0]] > odds_limit)):
            for i in range(2):              
                if pl_bet_lst[i]:
                    if row['pred_result'] != row['result']:
                        profit_lst.append(-betting_unit)
                        lose_count += 1
                    else:
                        profit = (betting_unit*row[odds_lst[i]])-betting_unit
                        profit_lst.append(profit)
                        win_count += 1
                else:
                    continue
        else:
            profit_lst.append(0)
    return [profit_lst, 'lr_probability_fixed', win_count, lose_count]

In [5]:
def lr_probability_martingale(df, betting_unit, arb1=0, arb2=0):
    profit_lst = []
    original_betting_unit = betting_unit
    win_count = 0
    lose_count = 0
    pl_odds = ['pl2_avg_bookmaker_odds', 'pl1_avg_bookmaker_odds']
    for index, row in df.iterrows():
        if ((row['pl1_prob'] > 0.48) and (row['pl1_prob'] < 0.52)):
            if row['pred_result'] != row['result']:
                profit_lst.append(-betting_unit)
                betting_unit *= 2
                lose_count += 1
            else:
                odds = pl_odds[row['result']]
                profit_lst.append((betting_unit*row[f'{odds}'])-betting_unit)
                betting_unit = original_betting_unit
                win_count += 1
        else:
            profit_lst.append(0)
    return [profit_lst, 'lr_probability_martingale', win_count, lose_count]

In [359]:
def result_fixed(df, betting_unit, model, par1=0):
    profit_lst = []
    win_count = 0
    lose_count = 0
    func_name = 'lr_result_fixed' if 'pred' in model else 'bm_result_fixed'
    pl_odds = ['pl2_avg_bookmaker_odds', 'pl1_avg_bookmaker_odds']
    for index, row in df.iterrows():
        if row[f'{model}'] != row['result']:
            profit_lst.append(-betting_unit)
            lose_count += 1
        else:
            odds = pl_odds[row['result']]
            profit_lst.append((betting_unit*row[f'{odds}'])-betting_unit)
            win_count += 1
    return [profit_lst, func_name, win_count, lose_count]

In [2]:
def result_martingale(df, betting_unit, model, par1=0):
    profit_lst = []
    original_betting_unit = betting_unit
    win_count = 0
    lose_count = 0
    func_name = ('lr_result_martingale'
                 if 'pred' in model 
                 else 'bm_result_martingale')
    pl_odds = ['pl2_avg_bookmaker_odds', 'pl1_avg_bookmaker_odds']
    for index, row in df.iterrows():
        if row[f'{model}'] != row['result']:
            profit_lst.append(-betting_unit)
            betting_unit *= 2
            lose_count += 1
        else:
            odds = pl_odds[row['result']]
            profit_lst.append((betting_unit*row[f'{odds}'])-betting_unit)
            betting_unit = original_betting_unit
            win_count += 1
    return [profit_lst, func_name, win_count, lose_count]